In [19]:
# load and optionally install required packages
if (!require('pacman')) install.packages('pacman')
pacman::p_load(
    dplyr,
    stringr,
    readr
)

In [20]:
# initial read in of all data
csv_files <- list.files(path='./data/generated', full.names = T, pattern = 'data_with_tokenized_no_rts_*')
data_groups <- lapply(csv_files, read_csv)
data.raw <- do.call(rbind, data_groups)

csv_files <- list.files(path='./data/generated', full.names = T, pattern = 'cluster_keywords_no_rts_*')
data_groups <- lapply(csv_files, read_csv)
keywords.raw <- do.call(rbind, data_groups)

Rows: 2447 Columns: 52
── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (32): content, domain_url, images.url, lang, matched_profile, noise_cate...
dbl (19): engagement, fluency_level, porn_level, reach, sentiment, url_views...
lgl  (1): title

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 24111 Columns: 52
── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (32): content, domain_url, images.url, lang, matched_profile, noise_cate...
dbl (19): engagement, fluency_level, porn_level, reach, sentiment, url_views...
lgl  (1): title

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Speci


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 10 Columns: 4
── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (2): cluster_name, keywords
dbl (2): cluster_number, docs

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 10 Columns: 4
── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (2): cluster_name, keywords
dbl (2): cluster_number, docs

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [21]:
keywords.raw <- keywords.raw %>% filter(!is.na(keywords))

In [22]:
keywords <- keywords.raw %>%
mutate(
    keywords = str_split(keywords, ' ')
)
data <- data.raw %>%
mutate(
    tokens = str_split(tokenized, ' ')
)


In [23]:
data.guessed <- data %>%
merge(keywords, by.x = 'cluster', by.y = 'cluster_name') %>%
rowwise() %>%
mutate(
    matched_keywords = intersect(as.list(tokens), as.list(keywords)) %>% paste(collapse=','),
    matched_keywords_count = intersect(as.list(tokens), as.list(keywords)) %>% length()
) %>%
filter(matched_keywords_count > 0) %>%
group_by(content) %>%
top_n(1, matched_keywords_count) %>%
arrange(desc(matched_keywords_count)) %>%
select(content, keywords, tokens, matched_keywords, matched_keywords_count, cluster, cluster_number) %>%
mutate(
    id = paste0(cluster, '_', cluster_number)
)
data.guessed

content,keywords,tokens,matched_keywords,matched_keywords_count,cluster,cluster_number,id
<chr>,<list>,<list>,<chr>,<int>,<chr>,<dbl>,<chr>
"Blessed is he whose help is in the God of Jacob, whose hope is in the Lord his God, who made heaven and earth, the sea, and all that is in them, who keeps faith forever; who executes justice for the oppressed, who gives food to the hungry. #Psalm146","god , whose , justic, food , help , keep , hope , jacob , lord , made , heaven, earth , sea , faith , forev","bless , whose , help , god , jacob , whose , hope , lord , god , made , heaven , earth , sea , keep , faith , forev , execut , justic , oppress, give , food , hungri , #psalm","whose,help,god,jacob,hope,lord,made,heaven,earth,sea,keep,faith,forev,justic,food",15,access_justice,8,access_justice_8
"Happy are those whose help is the God of Jacob, whose hope is in the Lord their God, who made heaven and earth, the sea, and all that is in them; who keeps faith forever; who executes justice for the oppressed; who gives food to the hungry.","god , whose , justic, food , help , keep , hope , jacob , lord , made , heaven, earth , sea , faith , forev","happi , whose , help , god , jacob , whose , hope , lord , god , made , heaven , earth , sea , keep , faith , forev , execut , justic , oppress, give , food , hungri","whose,help,god,jacob,hope,lord,made,heaven,earth,sea,keep,faith,forev,justic,food",15,access_justice,8,access_justice_8
This retired baseball player built the largest Black-owned McDonald’s franchise operation. Now he’s suing the fast-food chain for its ‘racist’ policies.,"oper , now , racist , polici , retir , basebal , player , built , largest , blackown, mcdonald, franchis, hes , su , fastfood","retir , basebal , player , built , largest , blackown, mcdonald, franchis, oper , now , hes , su , fastfood, chain , racist , polici","retir,basebal,player,built,largest,blackown,mcdonald,franchis,oper,now,hes,su,fastfood,racist,polici",15,access_justice,49,access_justice_49
This retired baseball player built the largest Black-owned McDonald’s franchise operation. Now he’s suing the fast-food chain for its ‘racist’ policies.,"oper , now , racist , polici , retir , basebal , player , built , largest , blackown, mcdonald, franchis, hes , su , fastfood","retir , basebal , player , built , largest , blackown, mcdonald, franchis, oper , now , hes , su , fastfood, chain , racist , polici","retir,basebal,player,built,largest,blackown,mcdonald,franchis,oper,now,hes,su,fastfood,racist,polici",15,access_justice,49,access_justice_49
This retired baseball player built the largest Black-owned McDonald’s franchise operation. Now he’s suing the fast-food chain for its ‘racist’ policies.,"oper , now , racist , polici , retir , basebal , player , built , largest , blackown, mcdonald, franchis, hes , su , fastfood","retir , basebal , player , built , largest , blackown, mcdonald, franchis, oper , now , hes , su , fastfood, chain , racist , polici","retir,basebal,player,built,largest,blackown,mcdonald,franchis,oper,now,hes,su,fastfood,racist,polici",15,access_justice,49,access_justice_49
"Blessed is he whose help is the God of Jacob, whose hope is in the Lord his God, who made heaven and earth, the sea, and all that is in them, who keeps faith forever; who executes justice for the oppressed, who gives food to the hungry. (Psalm 146:5-7)","god , whose , justic, food , help , keep , hope , jacob , lord , made , heaven, earth , sea , faith , forev","bless , whose , help , god , jacob , whose , hope , lord , god , made , heaven , earth , sea , keep , faith , forev , execut , justic , oppress, give , food , hungri , psalm","whose,help,god,jacob,hope,lord,made,heaven,earth,sea,keep,faith,forev,justic,food",15,access_justice,8,access_justice_8
"He says 'they're both the dark side' I think there is choice Proto fascist Capitalist (R) party supported by pro life, pro guns factions & mainly neo-liberal (D) party shakily allied w/ pro govt action, Social justice pro

In [24]:
write_to_file = function(data.to_write){
    write_csv(data.to_write, paste0('./data/generated/tweets_by_cluster_no_rts_', unique(data.to_write$id), '.csv'))
    return(data.to_write)
}

keywords %>%
mutate(
    id = paste0(cluster_name, '_', cluster_number)
) %>% merge(data.guessed) %>%
group_by(id) %>%
do(write_to_file(.))

cluster_number,keywords,id,cluster_name,docs,content,tokens,matched_keywords,matched_keywords_count,cluster
<dbl>,<list>,<chr>,<chr>,<dbl>,<chr>,<list>,<chr>,<int>,<chr>
24,"food , racist , chines , groceri, racism , store , peopl , call , get , like , just , eat , diet , can , becaus",access_justice_24,access_justice,958,"As a foodie I am sad to see racism in this food magazine. It reflects our current culture, where racism is pervasive in the fashion industry, and in so many places that are being revealed more broadly now. Hoping the shining light on bias brings change.","foodi , sad , see , racism , food , magazin , reflect , current , cultur , racism , pervas , fashion , industri, mani , place , reveal , broad , now , hope , shine , light , bias , bring , chang","racism,food",2,access_justice
24,"food , racist , chines , groceri, racism , store , peopl , call , get , like , just , eat , diet , can , becaus",access_justice_24,access_justice,958,". said it was ""reckoning with racism"" while telling a Latinx critic to be grateful for what she gets. The bravery of in telling her story means everything. Those who tell our food stories should be paid equitably - to do otherwise is gatekeeping. #FLFA20","said , reckon , racism , tell , latinx , critic , grate , get , braveri , tell , stori , mean , everyth , tell , food , stori , paid , equit , otherwis, gatekeep, #flfa","racism,get,food",3,access_justice
24,"food , racist , chines , groceri, racism , store , peopl , call , get , like , just , eat , diet , can , becaus",access_justice_24,access_justice,958,And you wonder why you got fired from Food Network for being a RACIST.,"wonder , whi , got , fire , food , network, racist","food,racist",2,access_justice
24,"food , racist , chines , groceri, racism , store , peopl , call , get , like , just , eat , diet , can , becaus",access_justice_24,access_justice,958,Anyways. The white photographer is playing “Braid My Hair” by Mario on set and it feels diet racist.,"anyway , white , photograph, play , braid , hair , mario , set , feel , diet , racist","diet,racist",2,access_justice
24,"food , racist , chines , groceri, racism , store , peopl , call , get , like , just , eat , diet , can , becaus",access_justice_24,access_justice,958,"This pandemic humbles you...makes you aware you are not the top of the food chain. This pandemic is not racist- it will take and ravage anyone. 8 to 80, blind,crippled or crazy. It takes the weak,healthy,the innocent. It holds no political party. It has made us weep.","pandem , humbl , youmak , awar , top , food , chain , pandem , racist , will , take , ravag , anyon , blindcrippl , crazi , take , weakhealthyth, innoc , hold , polit , parti , made , us , weep","food,racist",2,access_justice
24,"food , racist , chines , groceri, racism , store , peopl , call , get , like , just , eat , diet , can , becaus",access_justice_24,access_justice,958,They don't own hint or right?? I'm good then no racist Diet Coke for me!,"hint , right , good , racist, diet , coke","racist,diet",2,access_justice
24,"food , racist , chines , groceri, racism , store , peopl , call , get , like , just , eat , diet , can , becaus",access_justice_24,access_justice,958,"Man this Chinese food is good! Oops my bad, I meant COVID19 food. Don't wanna be racist.","man , chines, food , good , oop , bad , meant , covid , food , wanna , racist","chines,food,racist",3,access_justice
24,"food , racist , chines , groceri, racism , store , peopl , call , get , like , just , eat , diet , can , becaus",access_justice_24,access_justice,958,People of Color are at Greater Risk of COVID-19. Systemic Racism in the Food System Plays a Role. via,"peopl , color , greater, risk , covid , system , racism , food , system , play , role , via","peopl,racism,food",3,access_justice
24,"food , racist , chines , groceri, racism , store , peopl , call , get , like , just , eat , diet , can , becaus",access_justice_24,access_justice,958,something about the keto diet fee